In [ ]:
import os, time, warnings
warnings.filterwarnings( "ignore" )
# from IPython.core.display import HTML
from IPython.display import display, HTML

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## This is the correct way to use the random number generator,
##  since it allows finer control.
rand = np.random.RandomState( 321 )

# _EM_ and NIST

Before proceeding, it seems pedagogically necessary (at least for myself) to revise the **EM**-slgorithm and show its "correctness", so to say.

The $\TeX$ markup used here uses the "align*" environment and thus should not be viewed though nbViewer.

## A brief description of the **EM** algorithm

The EM algorithm seeks to maximize the likelihood by means of successive application of two steps: the E-step and the M-step.

For any probability measure $Q$ on the space of latent variables $Z$ with density $q$ the following holds:  
\begin{align*}
\log p(X|\Theta)
    &= \int q(Z) \log p(X|\Theta) dZ
     = \mathbb{E}_q \log p(X|\Theta) \\
   %% &= \Bigl[p(X,Z|\Theta) = p(Z|X,\Theta) p(X|\Theta) \Bigr] \\
    &= \mathbb{E}_{Z\sim q} \log \frac{p(X,Z|\Theta)}{p(Z|X\Theta)}
     = \mathbb{E}_{Z\sim q} \log \frac{q(Z)}{p(Z|X,\Theta)}
     + \mathbb{E}_{Z\sim q} \log \frac{p(X,Z|\Theta)}{q(Z)} \\ 
    &= KL\bigl(q\|p(\cdot|X,\Theta)\bigr) + \mathcal{L}\bigl(q, \Theta\bigr)
\end{align*}  

since the Bayes theorem posits that $p(X,Z|\Theta) = p(Z|X,\Theta) p(X|\Theta)$. Call this equiation the **"master equation"**.

Now note that since the Kullback-Leibler divergence is always non-negative, one has the following inequality:
$$\log p(X|\Theta) \geq \mathcal{L}\bigl(q, \Theta\bigr)$$

Let's try to make the lower bound as large as possible by changing $\Theta$ and varying $q$. But first note that the 
left-hand side of the **master equation** is independent of $q$, whence maximization of $\mathcal{L}$ with respect to $q$ (with $\Theta$ fixed) is equivalent to minimization of $KL\bigl(q\|p(\cdot|X,\Theta)\bigr)$ with respect to $q$ taking $\Theta$ fixed. Since $q$ is arbitrary, the optimal minimizer $q^*_\Theta$ is $q^*(Z|\Theta) = p(Z|X,\Theta)$ for all $Z$.

Now at the optimal distributuion $q^*_\Theta$ the **master equation** becomes
$$ \log p(X|\Theta)
= \mathcal{L}\bigl(q^*_\Theta, \Theta\bigr)
= \mathbb{E}_{Z\sim q^*_\Theta} \log \frac{p(X,Z|\Theta)}{q^*(Z|\Theta)}
= \mathbb{E}_{Z\sim q^*_\Theta} \log p(X,Z|\Theta) - \mathbb{E}_{Z\sim q^*_\Theta} \log q^*(Z|\Theta)
$$
for any $\Theta$. Thus the problem of log-likelihood maximization reduces to that of maximizing the sum of expectations on the right-hand side.

This new problem does not seem to be tractable in general since the optimization paramters $\Theta$ affect both the expected log-likelihood $\log p(X,Z|\Theta)$ under $Z\sim q^*_\Theta$ and the entropy of the optimal distribution of the latent variables $Z$.

Hopefully using an iterative procedure which switches between the computation of $q^*_\Theta$ and the maximization of $\Theta$ might be effective. Consider the folowing :
* **E**-step: considering $\Theta_i$ as given and fixed find $q^*_{\Theta_i} = \mathop{\text{argmin}}_q KL\bigl(q\|p(\cdot|X,\Theta_i)\bigr)$ and set $q_{i+1} = q^*_{\Theta_i}$;
* **M**-step: considering $q_{i+1}$ as given, solve $\mathcal{L}(q_{i+1},\Theta) \to \mathop{\text{max}}_\Theta$, where 
$$ \mathcal{L}(q,\Theta) = \mathbb{E}_{Z\sim q} \log p(X,Z|\Theta) - \mathbb{E}_{Z\sim q} \log q(Z) $$

The fact that $q_i$ is considered fixed makes the optimization of $\mathcal{L}(q_i,\Theta)$ equivalent to maximization of the expected log-likelihood, since the entropy term is fixed. Therefore the **M**-step becomes:
* given $q_{i+1}$ find $\Theta^*_{i+1} = \mathop{\text{argmax}}_\Theta \mathbb{E}_{Z\sim q_{i+1}} \log p(X,Z|\Theta)$ and put $\Theta_{i+1} = \Theta^*_{i+1}$.

Now, if the latent variables are mutually independent, then the optimal $q$ must be factorizable into marginal densities and:

\begin{align*}
    KL\bigl(q\|p(\cdot|X,\Theta)\bigr)
    &= \mathbb{E}_{Z\sim q} \log q(Z) - \sum_j \mathbb{E}_{z_j\sim q_j} \log p(z_j|X,\Theta)\\
    &= \sum_j \mathbb{E}_{z_j\sim q_j} \log q_j(z_j) - \sum_j \mathbb{E}_{z_j\sim q_j} \log p(z_j|X,\Theta)
     = \sum_j KL\bigl(q_j\|p_j(|X,\Theta)\bigr)
\end{align*}
where $q_j$ is the marginal desity of $z_j$ in $q(Z)$ (the last term in the first line comes from the Fubini theorem).

Therefore the **E**-step could be reduced to a set of minimization problems with respect to one-dimensional density functions:
$$ q_j^* = \mathop{\text{argmin}}_{q_j} KL\bigl(q_j\|p_j(\cdot|X,\Theta)\bigr) $$
since the Kulback-Leibler divergence in this case in additively separable.

### Correctness

Recall that the **master equation** is an identity: for all densities $q$ on $Z$ and for all admissible parameters $\Theta$
$$ \log p(X|\Theta) = KL\bigl(q\|p(\cdot|X,\Theta)\bigr) + \mathcal{L}\bigl(q, \Theta\bigr) $$

Hence if after the **E**-step the Kulback-Leibler divergence is reduced:
$$ KL\bigl(q'\|p(\cdot|X,\Theta)\bigr) \leq KL\bigl(q\|p(\cdot|X,\Theta)\bigr) $$
then for the same set of parameters $\Theta$ one has
$$ \mathcal{L}(q,\Theta) \leq \mathcal{L}(q',\Theta) $$

Just after the **E**-step one has $q_{i+1} = p(Z|X,\Theta_i)$, whence $KL\bigl(q_{i+1}\|p(\cdot|X,\Theta_i)\bigr) = 0$. In turn, this implies via the **master equation** that the following equality holds:
$$ \log p(X|\Theta_i) = \mathcal{L}(q_{i+1},\Theta_i) $$

After the **M**-step, since $\Theta_{i+1}$ is a maximizer, or at least an "_improver_" of $\mathcal{L}(q_{i+1},\Theta)$ compared to its value at $(q_i,\Theta_i)$, one has
$$ \mathcal{L}(q_{i+1},\Theta_i) \leq \mathcal{L}(q_{i+1},\Theta_{i+1}) $$

Threfore the effect of a single complete round of **EM** on the log-likelihood itself is:
$$ \log p(X|\Theta_i) = \mathcal{L}(q_{i+1},\Theta_i) \leq \mathcal{L}(q_{i+1},\Theta_{i+1}) \leq \mathcal{L}(q_{i+2},\Theta_{i+1}) = \log p(X|\Theta_{i+1}) $$
where the equality is achieved between the **E** and the **M** step within one round. This implies that **EM** indeed iteratively improves the log-likihood.

Note, that in the general case, without attaining zero Kulback-Leibler divergence at the $E$-step, one cannot be sure that the real log-likelihood is improved by each iteration and one can just say that
$$\mathcal{L}(q_{i+1},\Theta_i) \leq \log p(X|\Theta_i)$$
which does not uncover a relationship with $\log p(X|\Theta_{i+1})$. And without the guarantee that **EM** improves the log-likelihood to the maximum one cannot be sure about the consistency of the estimators. The key question is whether the lower bound $\mathcal{L}(q,\Theta)$ is any good.

## Application of the EM to NIST data

Each image is a random element in a discrete probability space $\Omega = \{0,1\}^{N\times M}$ with product-measure
$$\mathbb{P}(\omega) = \prod_{i=1}^N\prod_{j=1}^M \theta_{ij}^{\omega_{ij}} (1-\theta_{ij})^{1-\omega_{ij}}$$
for any $\omega\in \Omega$. In particular $M=N=28$. Basically each bit of the image is independent of any other bit and each one is a Bernoulli random variable with parameter $\theta_{ij}$: $\omega_{ij}\sim \text{Bern}(\theta_{ij})$.

Let's apply the EM algorithm to this dataset. The proposed model is the following.

Consider a mixture model of discrete probability spaces. Suppose there are $K$ componets in the mixture. Then each image is distributed according to the following law:
$$p(\omega|\Theta)
= \sum_{k=1}^K \pi_k p_k(\omega|\theta_k)
= \sum_{k=1}^K \pi_k \prod_{i=1}^N \prod_{j=1}^M \theta_{kij}^{\omega_{ij}} (1-\theta_{kij})^{1-\omega_{ij}}$$
where $\theta_{kij}$ is the paramter of the probability distribution of the $(i,j)$-th random variable (pixel) in the $k$-th class, and $\pi_k$ is the (prior) porbability of the $k$-th mixutre to generate a random element, $\sum_{k=1}^K \pi_k= 1$.

Suppose $X=(x_i)_{i=1}^n \in \Omega^n$ is the dataset. The log-likelihood is given by
$$ \log p(X|\Theta) = \sum_{s=1}^n \log \sum_{k=1}^K \pi_k
\prod_{i=1}^N \prod_{j=1}^M \theta_{kij}^{x_{sij}} (1-\theta_{kij})^{1-x_{sij}}$$
where $x_{sij}\in\{0,1\}$ -- is the value of the the $(i,j)$-th pixel at the $s$-th observation.

If the source $Z=(z_i)_{i=1}^n$ components of the mixture at each datapoint were known, then the log-likelihood would have been
$$ \log p(X,Z|\Theta) = \sum_{s=1}^n \log \prod_{k=1}^K \Bigl[ \pi_k 
\prod_{i=1}^N \prod_{j=1}^M \theta_{kij}^{x_{sij}} (1-\theta_{kij})^{1-x_{sij}} \Bigr]^{1_{z_s = k}}$$
where $1_{z_s = k}$ is the indicator and take the value $1$ if $\{z_s = k\}$ and $0$ otherwise ($1_{\{k\}}(z_s)$ is another notation).

The log-likelihood simplifies to
$$ \log p(X,Z|\Theta) = \sum_{s=1}^n \sum_{k=1}^K 1_{z_s = k} \Bigl( \log \pi_k + 
\sum_{i=1}^N \sum_{j=1}^M \bigl( x_{sij} \log \theta_{kij} + (1-x_{sij}) \log (1-\theta_{kij}) \bigr) \Bigr) $$
and further into a more separable form
$$ \log p(X,Z|\Theta)
= \sum_{s=1}^n \sum_{k=1}^K 1_{z_s = k} \log \pi_k
+ \sum_{s=1}^n \sum_{k=1}^K 1_{z_s = k} \Bigl( \sum_{i=1}^N \sum_{j=1}^M x_{sij} \log \theta_{kij}
+ \sum_{i=1}^N \sum_{j=1}^M (1-x_{sij}) \log (1-\theta_{kij}) \Bigr)$$

The expected log-likelihood under $z_s\sim q_s$ with $\mathbb{P}(z_s=k|X) = q_{sk}$, is given by
$$ \mathbb{E}\log p(X,Z|\Theta)
= \sum_{s=1}^n \sum_{k=1}^K q_{sk} \log \pi_k
+ \sum_{s=1}^n \sum_{k=1}^K q_{sk} \sum_{i=1}^N \sum_{j=1}^M \bigl( x_{sij} \log \theta_{kij} + (1-x_{sij}) \log (1-\theta_{kij}) \bigr)$$

#### Analytic solution

At the **E**-step one must compute $q^*(Z) = \mathbb{P}(z_s=k|X) = \hat{q}_{sk}$ based on the value of $\Theta = ((\pi_k), (\theta_{kij}))$.
$$\hat{q}_{sk}
= \frac{p(x_s|z_s=k,\Theta) p(z_s=k)}{\sum_{l=1}^K p(x_s|z_s=l,\Theta) p(z_s=l)}
\propto \pi_k \prod_{i=1}^N \prod_{j=1}^M \theta_{kij}^{x_{sij}} (1-\theta_{kij})^{1-x_{sij}}
$$
and
$$ q^*(Z) = \prod_{s=1}^n q_{s z_s} $$

In order to improve numerical stability and avoid numerical underflow it is better to use the following procedure for computation of the conditional probability:
$$ l_{sk} = \log \pi_k + \sum_{i=1}^N \sum_{j=1}^M \log \bigl( \theta_{kij}^{x_{sij}} (1-\theta_{kij})^{1-x_{sij}} \bigr)$$
set $l^*_s = \max_k l_{sk}$ and compute the log-sum
$$ \hat{l}_s = l^*_s + \log \sum_{k=1}^K \text{exp}\Bigl\{ l_{sk} - l^*_s \Bigr\} $$
This seemingly redunant subctration and addition of $l^*_s$ helps avoid underflow during the numerical exponentiation.
However if the underflow still took place in the sum-exp, then those values of $\hat{l}_s$ that evaluated to floating point _NAN_ can reliably be replaced with respective values of $l^*_s$.  After this sanitization the **E**-step's optimal distribution is calculated as:
$$ \hat{q}_{sk} = \text{exp}\Bigl\{ l_{sk} - \hat{l}_s \Bigr\} $$
which would yield a numerically accurate distribution.

If $l^*_s >> l_{sk}$ for all $k$ but such that $l^*_s = l_{sk}$ (let it be $k^*$), then an underflow occurs at the sum-exp step, whence for some very small $\epsilon > 0$ one has
$$\hat{l}_s = l^*_s + \log (1+\epsilon) $$
whence
$$\hat{q}_{sk} = \text{exp}\bigl\{l_{sk} - \hat{l}_s\bigr\} = (1+\epsilon)^{-1} \cdot \text{exp}\bigl\{l_{sk} - l^*_s \bigr\}$$
For $k=k^*$ on has $\hat{q}_{sk} = \frac{1}{1+\epsilon}\approx 1$, and for $k\neq k^*$ -- $\hat{q}_{sk} = \frac{\eta}{1+\epsilon} \approx 0$ for some extremely small $\eta>0$.

The variables in the code have the following dimensions:
* $\theta \in [0,1]^{K\times (N\times M)}$;
* $\pi \in [0,1]^{1\times K}$;
* $x \in \{0,1\}^{n\times (N\times M)}$;
* $z \in [0,1]^{n\times K}$.

In [ ]:
## A bunch of wrappers to match the task specifications
def posterior( x, clusters ) :
    pi = np.ones( clusters.shape[ 0 ], dtype = np.float ) / clusters.shape[ 0 ]
    q, ll = __posterior( x, theta = clusters, pi = pi )
    return q

## The likelihood is a byproduct of the E-step's minimization of Kullback-Leibler
def likelihood( x, clusters ) :
    pi = np.ones( clusters.shape[ 0 ], dtype = np.float ) / clusters.shape[ 0 ]
    q, ll = __posterior( x, theta = clusters, pi = None )
    return ll

## The core procedure for computing the conditional density of classes
def __posterior( x, theta, pi ) :
## Unfortunately sometimes there can be negative machine zeros, which
##  spoil the log-likelihood computation by poisoning with NANs.
## That is why the theta array is restricted to [0,1].
    theta_clipped = np.clip( theta, 0.0, 1.0 )
## Note that the power formulation is just a mathematically convenient
##  way of writing \theta if x=1 or (1-\theta) otherwise. This comapct
##  way of computing had to be abandoned, since it required huge amounts
##  of memory.
#     ll_skij = np.where( x[:, np.newaxis] > 0,
#         np.log( theta_clipped ), np.log( 1 - theta_clipped ) )
#     ll_sk = np.sum( ll_skij, axis = ( 2, ) ) + np.log( pi )
#     del ll_skij
## Iterate over classes
    ll_sk = np.zeros( ( x.shape[ 0 ], theta.shape[ 0 ] ), dtype = np.float )
    for k in xrange( theta.shape[ 0 ] ) :
        ll_sij = np.where( x > 0, np.log( theta_clipped[ k ] ), np.log( 1 - theta_clipped[ k ] ) )
        ll_sk[ :, k ] = np.sum( ll_sij, axis = ( 1, ) ) + np.log( pi[ k ] )
## Find the largest unnormalized probability.
    llstar_s = np.reshape( np.max( ll_sk, axis = ( 1, ) ), ( ll_sk.shape[ 0 ], 1 ) )
## Compute the log-sum-exp of the individual log-likelihoods
    ll_s = np.reshape( np.log( np.sum( np.exp( ll_sk - llstar_s ), axis = ( 1, ) ) ), ( ll_sk.shape[ 0 ], 1 ) )
## The sum-exp could never be anything lower than 1, since at least one
##  element of each row of ll_sk has to be lstar_s, whence the respective
##  difference should be zero and the exponent -- 1. Thus even if the
##  rest of the sum is close to machine zero, the logarithm would still
##  return 0.
    ll_s += llstar_s
    del llstar_s
## Normalise the likelihoods to get conditional probability, and compute
##  the sum of the log-denominator, which is the log-likelihood.
    return np.exp( ll_sk - ll_s ), np.sum( ll_s )

At the **M**-step for some fixed $q(Z)$ one solves $\mathbb{E}\log p(X,Z|\Theta)\to \max_\Theta$ subject to $\sum_{k=1}^K \pi_k = 1$ which is a convex optimization problem with respect to $\Theta$, since the log-likelihood as a linear combination of convex functions is convex. The first order condition is $\sum_{s=1}^n \frac{q_{sk}}{\pi_k} - \lambda = 0$ for all $k=1,\ldots,K$, whence $ \lambda = \sum_{s=1}^n \sum_{l=1}^K q_{sl} = n $ and finally
$$\hat{\pi}_k = \frac{\sum_{s=1}^n q_{sk}}{n}$$
For $\theta_{kij}$, $i=1,\ldots,N$, $j=1,\ldots,M$ and $k=1,\ldots,K$ the FOC is
$$ \sum_{s=1}^n q_{sk} \frac{x_{sij}}{\theta_{kij}} - \sum_{s=1}^n q_{sk} \frac{1-x_{sij}}{1-\theta_{kij}} = 0 $$
whence
$$\hat{\theta}_{kij} =  \frac{\sum_{s=1}^n q_{sk} x_{sij}}{ \sum_{s=1}^n q_{sk} } = \frac{\sum_{s=1}^n q_{sk} x_{sij}}{ n \hat{\pi}_k }$$


In [ ]:
## A wrapper to match the task specifications
def learn_clusters( x, z ) :
    theta, pi = __learn_clusters( x, z )
    return theta

## The E-step is simple: just compute the optimal parameters under
##  the current conditional distribution of the latend variables.
def __learn_clusters( x, z ) :
## The prior class probabilities
    pi = np.sum( z, axis = ( 0, ) )
## Pixel probabilities conditional on the calss
    theta = np.dot( z.T, x ) / np.reshape( pi, ( z.shape[ 1 ], 1 ) )
## Return
    return theta, np.reshape( pi, ( 1, pi.shape[ 0 ] ) ) / np.sum( pi )

A it has been mentioned eariler, the EM algorithm switches between **E** and **M** steps until convergence.

In [ ]:
## A wrapper for the core em algorithm above
def em_algorithm( x, K, maxiter, verbose = True, rel_eps = 1e-4 ) :
## Initialize the model parameters with uniform [0,1] random numbers
    theta = rand.uniform( size = ( K, x.shape[ 1 ] ) )
## Run the em algorithm
    ll, theta, pi, status = __em_algorithm( x, theta_1 = theta,
        pi_1 = None, niter = maxiter, rel_eps = rel_eps, verbose = verbose )
## Return the history of theta and the final log liklihood
    if verbose :
        if status[ 'status' ] != 0 :
            print "Convergence not achieved. %d" % ( status[ 'status' ], )
    return theta, ll

The procedure above actually invokes the true EM core, defined below.

In [ ]:
## The core of the EM algorithm
def __em_algorithm( x, theta_1, pi_1 = None, niter = 1000, rel_eps = 1e-4, verbose = True ) :
## If we were supplied with an initial estimate of the prior distribution,
##  then assume the full model is needed.
    full_model = pi_1 is not None
## If the prior cluster probabilities are not supplied, assume uniform distribution.
    pi_1 = pi_1 if pi_1 is not None else np.ones( theta_1.shape[ 0 ], dtype = np.float ) / theta_1.shape[ 0 ]
## Allocate the necessary space for the history of model estimates
    theta_hist = np.empty( ( 0, ) + theta_1.shape, dtype = np.float )
    pi_hist = np.empty( ( 0, theta_1.shape[ 0 ] ), dtype = np.float )
    ll_hist = np.empty( 0, np.float )
## Set "old" estimated to zero. At this line the current estimates are in fact
##  the initially provided ones.
    theta_0, pi_0 = np.zeros_like( theta_1 ), np.zeros_like( pi_1 )
## Initialize the loop
    status, kiter, rel_theta, rel_pi, ll = -1, 0, np.nan, np.nan, -np.inf
    while kiter < niter :
## Dump the current estimators and other information.
        if verbose :
            print( "Iteration %d: avg. log-lik: %.3f, $\\Theta$ div. %.3f, $\\Pi$ div. %.3f" % (
                kiter, ll / x.shape[ 0 ], rel_theta, rel_pi ) )
            show_data( theta_1 - theta_0 if True else theta_1, n = theta_0.shape[ 0 ],
                n_col = min( 10, theta_0.shape[ 0 ] ), cmap = plt.cm.hot, interpolation = 'nearest' )
## The convergence criterion is the L^∞ norm of relative L^1 errors
        if max( rel_pi, rel_theta ) < rel_eps :
            status = 0
            break ;
## Overwrite the initial estimates
        theta_0, pi_0 = theta_1, pi_1
## E-step: call the core posterior function to get both the log-likelihood
##  and the estimate of the conditional distribution.
        z_1, ll = __posterior( x, theta_0, pi_0 )
## M-step: compute the optimal parameters under the current estimate of the posterior
        theta_1, pi_1 = __learn_clusters( x, z_1 )
## Discard the computed estimate of pi if the model is discriminative (conditional likelihood).
        if not full_model :
            pi_1 = pi_0
## Record the current estimates to the history
        theta_hist = np.vstack( ( theta_hist, theta_1[np.newaxis] ) )
        pi_hist = np.vstack( ( pi_hist, pi_1 ) )
        ll_hist = np.append( ll_hist, ll )
## Check for bad float numbers
        if not ( np.all( np.isfinite( theta_1 ) ) and np.all( np.isfinite( pi_1 ) ) ) :
            status= -2
            break ;
## Check convergence: L^1 relative error. If the relative margin is exactly
##  zero, then return NaNs. This makes teh loop exhaust all iterations, since
##  any comprison agains a NaN returns False.
        rel_theta = np.sum( np.abs( theta_1 - theta_0 ) / ( np.abs( theta_0 ) + rel_eps ) ) if rel_eps > 0 else np.nan
        rel_pi = np.sum( np.abs( pi_1 - pi_0 ) / ( np.abs( pi_0 ) + rel_eps ) ) if rel_eps > 0 else np.nan
## Next iteration
        kiter += 1
    return ll_hist, theta_hist, pi_hist, { 'status': status, 'iter': kiter }

### Miscellanea

In order to be able to plot more flexibly, define another arranger.

In [ ]:
## A more flexible image arrangement
def arrange_flex( images, n_row = 10, n_col = 10, N = 28, M = 28, fill_value = 0 ) :
## Create the final grid of images row-by-row
    im_grid = np.full( ( n_row * N, n_col * M ), fill_value, dtype = images.dtype )
    for k in range( min( images.shape[ 0 ], n_col * n_row ) ) :
## Get the grid cell at which to place the image
        i, j = ( k // n_col ) * N, ( k % n_col ) * M
## Just put the image in the cell
        im_grid[ i:i+N, j:j+M ] = np.reshape( images[ k ], ( N, M, ) )
    return im_grid

The procedure below loads the MNIST data from a comma-separated text file.

In [ ]:
def load_mnist_from_csv( filename ) :
## Read the CSV file
    data = np.loadtxt( open( filename, "rb" ), dtype = np.short, delimiter = ",", skiprows = 0 )
## Peel off the lables
    return data[:,1:], data[:,0]

A procedure for pickling numpy arrays of floats.

In [ ]:
import base64, zlib

def np_pack( data ) : return base64.b64encode( zlib.compress( data ) )
def np_unpack( text ) : return np.frombuffer( zlib.decompress(
    base64.decodestring( text ) ), dtype = np.float )

A handy procedure for one-line visualization.

In [ ]:
def show_data( data, n, n_col = 10, **kwargs ) :
## Get the number of rows necessary to plot the needed number of images
    n_row = ( n + n_col - 1 ) // n_col
## Set the dimensions of the figure
    plt.figure( figsize = ( n_col, n_row ) )
## Plot!
    plt.imshow( arrange_flex( data[:n], n_col = n_col, n_row = n_row ), **kwargs )
    plt.show( )

Define a convenient procedure for running experiments. By setting relative error to zero the algorithm is forced to exhaust all the allocated iterations.

In [ ]:
def experiment( data, K, maxiter, verbose = True, until_convergence = False ) :
## Run the EM
    return em_algorithm( data, K, maxiter, rel_eps = 1.0e-4 if until_convergence else 0.0, verbose = verbose )

def visualize( data, clusters, ll, n_col = 2, plot_ll = True ) :
## Display the result
    print "Final conditional log-likelihood value per observation achieved %f in %d iteration(s)" % ( ll[-1] / data.shape[ 0 ], len( ll ) )
## Plot the first difference of average log-likelihood
    if plot_ll :
        plt.figure( figsize = ( 16, 9 ) )
        ax = plt.subplot(111)
        ax.set_title( r"avg. log-likelihood change between successive iterations (log scale)" )
        ax.plot( np.diff( ll / data.shape[ 0 ]  ) )
        ax.set_ylabel( r"$\Delta_i \frac{1}{n} \sum_{s=1}^n \mathbb{E}_{z_s\sim q_i} \log p(x_s,z_s|\Theta_i)$" )
        ax.set_yscale( 'log' )
## Plot the final estimates
    if n_col > 0 :
        show_data( clusters[-1], n = clusters.shape[1], n_col = n_col, cmap = plt.cm.gray, interpolation = 'nearest' )

Define a function that produces and embed a video in HTML

In [ ]:
## Make simple animations of the EM estimatora
## http://jakevdp.github.io/blog/2013/05/12/embedding-matplotlib-animations/
## http://jakevdp.github.io/blog/2012/08/18/matplotlib-animation-tutorial/
from matplotlib import animation
from IPython.display import HTML
from tempfile import NamedTemporaryFile
VIDEO_TAG = """<video controls><source src="data:video/x-m4v;base64,{0}" type="video/mp4">Your browser does not support the video tag.</video>"""
def display_animation( anim ) :
    plt.close( anim._fig )
    if not hasattr( anim, '_encoded_video' ) :
        ffmpeg_writer = animation.FFMpegWriter( )
        with NamedTemporaryFile( suffix = '.mp4' ) as f:
            anim.save( f.name, fps = 15, extra_args = [ '-vcodec', 'libx264' ] )# , writer = ffmpeg_writer )
            video = open( f.name, "rb" ).read( )
        anim._encoded_video = video.encode( "base64" )
    return HTML( VIDEO_TAG.format( anim._encoded_video ) )

def animate( theta, ll, n_col = 10, n_row = 10, diff = True, interval = 20, **kwargs ) :
## First set up the figure, the axis, and the plot element we want to animate
    fig = plt.figure( )
## Create two subplots
    ax1, ax2 = fig.add_subplot( 211 ), fig.add_subplot( 212 )
## Set the background
    bg = arrange_flex( np.zeros_like( theta[ 0 ] ), n_col = n_col, n_row = n_row )
## Initialize different ranges for the image aritst
    if diff :
        im = ax1.imshow( bg, vmin = -1.0, vmax = +1.0, **kwargs )
    else :
        im = ax1.imshow( bg, vmin = +0.0, vmax = +1.0, **kwargs )
## Compute log-likelihood differences and initialize geomtery for the line artist
    ll_diff = np.diff( ll ) / data.shape[ 0 ]
    line, = ax2.plot( [], linestyle = "-", color = 'blue' )
    ax2.set_xlim( -0.1, theta.shape[ 0 ] + 0.1 )
    ax2.set_yscale( 'log' )
    ax2.set_ylim( np.min( ll_diff ) * 0.9, np.max( ll_diff ) * 1.1 )
## Animation function.  This is called sequentially
    def update( i ) :
## Compute the frame
        if diff :
            frame = theta[ i ] - theta[ i-1 ] if i > 0 else theta[ 0 ]
            frame /= np.max( np.abs( frame ) )
        else :
            frame = theta[ i ]
## Draw the frame on the image artist
        im.set_data( arrange_flex( frame, n_col = n_col, n_row = n_row ) )
        if i > 0 :
## Show history on the line artist
            line.set_data( np.arange( i ), ll_diff[ :i ] )
## Return an iterator of artists in this frame
        return im, line,
## Call the animator.
    return animation.FuncAnimation( fig, update, frames = theta.shape[ 0 ], interval = interval )

<hr/>

## Loading and visualizing MNIST

###Preamble

Obtain the MNIST data

In [ ]:
if False :
## Fetch MNIST dataset from SciKit and create a local copy.
    from sklearn.datasets import fetch_mldata
    mnist = fetch_mldata( "MNIST original", data_home = './data/' )
    np.savez_compressed('./data/mnist/mnist_scikit.npz', data = mnist.data, labels = mnist.target )

if False :
## Fetch the data from the provided CSV (!) files and save as a compressed data blob
    data, labels = load_mnist_from_csv( "./data/mnist/mnist_train.csv" )
    np.savez_compressed( './data/mnist/mnist_train.npz', labels = labels, data = data )
    data, labels = load_mnist_from_csv( "./data/mnist/mnist_test.csv" )
    np.savez_compressed( './data/mnist/mnist_test.npz', labels = labels, data = data )

###Study

First of all load and binarize the training data using the values 127 as the threshold.

In [ ]:
assert( os.path.exists( './data/mnist/mnist_train.npz' ) )
with np.load( './data/mnist/mnist_train.npz', 'r' ) as npz :
    mnist_labels, mnist_data = npz[ 'labels' ], np.array( npz[ 'data' ] > 127, np.int )

####  Case : $K=2$

Let's have a look at some 6s and 9s.

In [ ]:
## Mask
inx_sixes, inx_nines = np.where( mnist_labels == 6 )[ 0 ], np.where( mnist_labels == 9 )[ 0 ]
## Extract
sixes = mnist_data[ rand.choice( inx_sixes, 90, replace = False ) ]
nines = mnist_data[ rand.choice( inx_nines, 90, replace = False ) ]
## Show
show_data( sixes, n = 45, n_col = 15, cmap = plt.cm.gray, interpolation = 'nearest' )
show_data( nines, n = 45, n_col = 15, cmap = plt.cm.gray, interpolation = 'nearest' )

They do indeed look quite distinct. Now collect them into a single dataset and estimate the model.

In [ ]:
data = mnist_data[ np.append( inx_sixes, inx_nines ) ]
clusters, ll = experiment( data, 2, 30 )

The estimate deltas show that the **EM** algorithm's **E**-step actually transfers the unlikely observations between classes, as is expected by constructon of the algorithm.

Judging by the plot below, it turns out that 30 iterations is more than enough for the **EM** to get meaninful estimates the class ideals, represented by the probability porduct-measure on $\Omega^{28\times 28}$.

In [ ]:
visualize( data, clusters, ll )

Now let's see how well the **EM** algorithm performs on a model with more classes. But before that let's have a look at a random sample of the handwritten digits.

In [ ]:
indices = np.arange( mnist_data.shape[ 0 ] )
rand.shuffle( indices )
plt.imshow( arrange_flex( mnist_data[ indices[:100] ] ), cmap = plt.cm.gray, interpolation = 'nearest' )
plt.show( )

#### Case : $K=10, 15, 20$

The original size of the trainig sample is too large to fit in these RAM banks :) That is why I had to limit the sample to a random subset of 2000 observations.

In [ ]:
data = mnist_data[ indices[:2000]]

Run the procedure that perfoems **EM** algorithm and return the history of the parameter estimates as well as the dynamics of the log-likelihood lower bonud.

In [ ]:
clusters, ll = experiment( data, 10, 50 )

One can clearly see, that $50$ iterations were not enough for the alogirithm to converge: though the changes are tiny, even on the log-scale, they are still unstable.

In [ ]:
visualize( data, clusters, ll, n_col = 5 )

Furthermore the returned templates of the mixture components and clearly suboptimal: the procedure seems to get stuck at individual examples. One way to remedy this is to use more iterations.

In [ ]:
clusters_10, ll_10 = experiment( data, 10, 1000, verbose = True, until_convergence = True )

As one can see, increasing the number of iterations does not necessarily improve the results.

In [ ]:
visualize( data, clusters_10, ll_10, n_col = 5 )

Let's see if changing $K$ does the trick.

In [ ]:
clusters_15, ll_15 = experiment( data, 15, 1000, verbose = True, until_convergence = True )

In [ ]:
clusters_20, ll_20 = experiment( data, 20, 1000, verbose = True, until_convergence = True )

In [ ]:
visualize( data, clusters_10, ll_10, n_col = 10, plot_ll = False )
visualize( data, clusters_15, ll_15, n_col = 10, plot_ll = False )
visualize( data, clusters_20, ll_20, n_col = 10, plot_ll = False )

In [ ]:
display_animation( animate( clusters_20, ll_20, 10, 2, diff = False, cmap = plt.cm.gray, interpolation = 'nearest' ) )

<hr/>

A random variable $X\sim \text{Beta}(\alpha,\beta)$ if the law of $X$ has density
$$p(u) = \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} u^{\alpha-1}(1-u)^{\beta-1} $$

$$ \log p(X,Z|\Theta) = \sum_{s=1}^n \log \prod_{k=1}^K \Bigl[ \pi_k 
\prod_{i=1}^N \prod_{j=1}^M 
\frac{\Gamma(\alpha_{kij}+\beta_{kij})}{\Gamma(\alpha_{kij})\Gamma(\beta_{kij})} x_{sij}^{\alpha_{kij}-1}(1-x_{sij})^{\beta_{kij}-1} \Bigr]^{1_{z_s = k}}$$

\begin{align*}
\mathbb{E}_q \log p(X,Z|\Theta)
&= \sum_{k=1}^K \sum_{s=1}^n q_{sk} \log \pi_k \\
&+ \sum_{k=1}^K \sum_{i=1}^N \sum_{j=1}^M \sum_{s=1}^n q_{sk} \bigl(
    \log \Gamma(\alpha_{kij}+\beta_{kij}) - \log \Gamma(\alpha_{kij}) - \log \Gamma(\beta_{kij}) \bigr) \\
&+ \sum_{k=1}^K \sum_{i=1}^N \sum_{j=1}^M \sum_{s=1}^n q_{sk} \bigl(
    (\alpha_{kij}-1) \log x_{sij} + (\beta_{kij}-1) \log(1-x_{sij}) \bigr) \\
\end{align*}


Derivative of a Gamma function does not seem to yeild analytically tracktable solutions.

<hr/>

#### Nonparametric approach

In [ ]:
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.grid_search import GridSearchCV

In [ ]:
pca = PCA( n_components = 50 )
X_train_pca = pca.fit_transform( X_train )

In [ ]:
params = { 'bandwidth' : np.logspace( -1, 1, 20 ) }
grid = GridSearchCV( KernelDensity( ), params )
grid.fit( X_train_pca )
print("best bandwidth: {0}".format( grid.best_estimator_.bandwidth ) )

In [ ]:
params
kde = grid.best_estimator_

In [ ]:
new_data = kde.sample( 100 )
new_data = pca.inverse_transform( new_data )
print new_data.shape

plt.figure( figsize = ( 9, 9 ) )
plt.imshow( arrange_flex( new_data ), cmap = plt.cm.gray, interpolation = 'nearest' )
plt.show( )